# Test EIA860 validation for old years

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import sys
import os
import pathlib

# 3rd party libraries
import geopandas as gpd
import dask.dataframe as dd
from dask.distributed import Client
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

In [3]:
# Enable viewing of logging outputs
logger=logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

In [4]:
# Display settings
sns.set()
%matplotlib inline
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 5

In [5]:
frequency = 'AS' # annual

In [6]:
# Establish connection to pudl database
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, freq=frequency) #annual frequency

In [7]:
plants = pudl_out.gens_eia860()

In [8]:
#pudl_out.plants_steam_ferc1()

In [25]:
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
# see all the tables inside of the database
pudl_engine.table_names()

['assn_gen_eia_unit_epa',
 'assn_plant_id_eia_epa',
 'boiler_fuel_eia923',
 'boiler_generator_assn_eia860',
 'boilers_entity_eia',
 'coalmine_eia923',
 'energy_source_eia923',
 'ferc_accounts',
 'ferc_depreciation_lines',
 'fuel_ferc1',
 'fuel_receipts_costs_eia923',
 'fuel_type_aer_eia923',
 'fuel_type_eia923',
 'generation_eia923',
 'generation_fuel_eia923',
 'generators_eia860',
 'generators_entity_eia',
 'ownership_eia860',
 'plant_in_service_ferc1',
 'plant_unit_epa',
 'plants_eia',
 'plants_eia860',
 'plants_entity_eia',
 'plants_ferc1',
 'plants_hydro_ferc1',
 'plants_pudl',
 'plants_pumped_storage_ferc1',
 'plants_small_ferc1',
 'plants_steam_ferc1',
 'prime_movers_eia923',
 'purchased_power_ferc1',
 'transport_modes_eia923',
 'utilities_eia',
 'utilities_eia860',
 'utilities_entity_eia',
 'utilities_ferc1',
 'utilities_pudl',
 'utility_plant_assn']

In [ ]:
gens = pd.read_sql("generators_eia860", pudl_engine)

In [17]:
gens

,id,plant_id_eia,generator_id,report_date,operational_status_code,operational_status,ownership_code,owned_by_non_utility,utility_id_eia,capacity_mw,reactive_power_output_mvar,summer_capacity_mw,winter_capacity_mw,summer_capacity_estimate,winter_capacity_estimate,energy_source_code_1,energy_source_code_2,energy_source_code_3,energy_source_code_4,energy_source_code_5,energy_source_code_6,energy_source_1_transport_1,energy_source_1_transport_2,energy_source_1_transport_3,energy_source_2_transport_1,energy_source_2_transport_2,energy_source_2_transport_3,fuel_type_code_pudl,distributed_generation,multiple_fuels,deliver_power_transgrid,syncronized_transmission_grid,turbines_num,planned_modifications,planned_net_summer_capacity_uprate_mw,planned_net_winter_capacity_uprate_mw,planned_uprate_date,planned_net_summer_capacity_derate_mw,planned_net_winter_capacity_derate_mw,planned_derate_date,planned_new_prime_mover_code,planned_energy_source_code_1,planned_repower_date,other_planned_modifications,other_modifications_date,planned_retirement_date,carbon_capture,startup_source_code_1,startup_source_code_2,startup_source_code_3,startup_source_code_4,technology_description,turbines_inverters_hydrokinetics,time_cold_shutdown_full_load_code,planned_new_capacity_mw,cofire_fuels,switch_oil_gas,nameplate_power_factor,minimum_load_mw,uprate_derate_during_year,uprate_derate_completed_date,current_planned_operating_date,summer_estimated_capability_mw,winter_estimated_capability_mw,retirement_date,data_source
0,1,63941,OA1,2019-01-01,U,proposed,S,None,63477.0,14.4,NaN,14.4,14.4,None,None,WND,None,None,None,None,None,None,None,None,None,None,None,wind,None,None,None,False,NaN,None,NaN,NaN,NaT,NaN,NaN,NaT,None,None,NaT,None,NaT,NaT,None,None,None,None,None,Onshore Wind Turbine,4.0,None,NaN,None,None,NaN,NaN,None,NaT,2020-12-01,NaN,NaN,NaT,eia860
1,2,63940,CEN40,2019-01-01,T,proposed,S,None,63606.0,40.0,NaN,40.0,40.0,None,None,SUN,None,None,None,None,None,None,None,None,None,None,None,solar,None,None,None,False,NaN,None,NaN,NaN,NaT,NaN,NaN,NaT,None,None,NaT,None,NaT,NaT,None,None,None,None,None,Solar Photovoltaic,None,None,NaN,None,None,NaN,NaN,None,NaT,2020-12-01,NaN,NaN,NaT,eia860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233982,233983,3,2,2009-01-01,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaT,NaN,NaN,NaT,None,None,NaT,None,NaT,NaT,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,NaT,NaT,NaN,NaN,NaT,None
233983,233984,3,1,2009-01-01,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaT,NaN,NaN,NaT,None,None,NaT,None,NaT,NaT,None,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,None,NaT,NaT,NaN,NaN,NaT,None
